# Web Scraping Exercise

## 1. Introduction and Planning

### Objective:
The goal of this exercise is to build a web scraper that collects data from a chosen website. You will learn how to send HTTP requests, parse HTML content, extract relevant data, and store it in a structured format.

### Tasks:
1. Identify the data you want to scrape.
2. Choose the target website(s).
3. Plan the structure of your project.

### Example:
For this exercise, we will scrape job listings from Indeed.com. We will extract job titles, company names, locations, and job descriptions.

## 2. Understanding the Target Website
### Objective:

Analyze the structure of the web pages to be scraped.
### Tasks:

* Inspect the target website using browser developer tools.
* Identify the HTML elements that contain the desired data.

### Instructions:

* Open your browser and navigate to the target website (e.g., Indeed.com).
* Right-click on the webpage and select "Inspect" or press Ctrl+Shift+I.
* Use the developer tools to explore the HTML structure of the webpage.
* Identify the tags and classes of the elements that contain the job titles, company names, locations, and descriptions.

## 3. Writing the Scraper
### Objective:

Develop the code to scrape data from the target website.
### Tasks:

* Send HTTP requests to the target website.
* Parse the HTML content and extract the required data.
* Handle pagination to scrape data from multiple pages.
* Implement error handling.

In [1]:
from IPython.display import display

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from tqdm import tqdm


In [3]:
headers = {
    "User-Agent": "My Python App"
}


In [4]:
file_path = r"C:\Users\Ronny Amores\Desktop\EPN\Octavo\RI\Deberes\Deberes_IIB\Webcrawling\Recipes A-Z _ Allrecipes.com.html"

with open(file_path, "r", encoding="utf-8") as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, "html.parser")


In [5]:
def obtenerLinks(htmlLinks):
    totalLinks = []
    for itemLink in htmlLinks.find_all("a", class_="mntl-card-list-items"):
        if itemLink.has_attr('href'):
            totalLinks.append(itemLink['href'])
    return totalLinks


In [6]:
def obtenerTitulo(HtmlTitulo):
    titulorespuesta = []
    for tituloreceta in HtmlTitulo:
        titulorespuesta.append(tituloreceta.find("title").text)
    return titulorespuesta


In [7]:
def descripcionReceta(HTMLReceta):
    respuestaDescripcion = []
    for descripcionReceta in HTMLReceta:
        try:
            resp = descripcionReceta.find("div", class_="summary_text").text.strip()
            respuestaDescripcion.append(resp)
        except AttributeError:
            respuestaDescripcion.append("")
    return respuestaDescripcion


In [8]:
def ingredientesReceta(HTMLIngrediente):
    ingredientes = []
    for ingredienteItem in HTMLIngrediente:
        respuestaIngredientes = ''
        for item in ingredienteItem.find_all("span", class_="ingredients-item-name"):
            respuestaIngredientes += item.text.strip() + '\n'
        ingredientes.append(respuestaIngredientes)
    return ingredientes


In [9]:
def instruccionesReceta(HTMLPasos):
    instrucciones = []
    for pasosItem in HTMLPasos:
        respuestaInstrucciones = ''
        for itemPaso in pasosItem.find_all("span", class_="recipe-directions__list--item"):
            respuestaInstrucciones += itemPaso.text.strip() + '\n'
        instrucciones.append(respuestaInstrucciones)
    return instrucciones


In [10]:
headers = {
    "User-Agent": "My Python App"
}


In [11]:
def responseURL(urlresponse, headers, timeout=10):
    html_content = []
    for url1 in tqdm(urlresponse, "Progreso"):
        try:
            response = requests.get(url1, headers=headers, timeout=timeout)
            htmltext = response.text
            html_content.append(BeautifulSoup(htmltext, "html.parser"))
        except requests.Timeout:
            print(f"Timeout occurred for URL: {url1}")
        except requests.RequestException as e:
            print(f"Request failed for URL: {url1} with exception: {e}")
    return html_content


In [13]:
# Obtener enlaces de las recetas
linksHtml = obtenerLinks(soup)
print("Enlaces obtenidos:", linksHtml)

# Crear DataFrame intermedio para ver los enlaces
df_links = pd.DataFrame({'Enlaces': linksHtml})
display(df_links)

# Obtener el contenido HTML de cada enlace
HTMLrecetas = responseURL(linksHtml, headers)

# Obtener los datos de las recetas
titulo_receta = obtenerTitulo(HTMLrecetas)
descripcion = descripcionReceta(HTMLrecetas)
ingredientes = ingredientesReceta(HTMLrecetas)
instrucciones = instruccionesReceta(HTMLrecetas)

# Crear DataFrame intermedio para ver los datos extraídos
df_titulos = pd.DataFrame({'Titulo': titulo_receta})
df_descripciones = pd.DataFrame({'Descripcion': descripcion})
df_ingredientes = pd.DataFrame({'Ingredientes': ingredientes})
df_instrucciones = pd.DataFrame({'Instrucciones': instrucciones})

display(df_titulos)
display(df_descripciones)
display(df_ingredientes)
display(df_instrucciones)

# Crear un DataFrame final con los datos obtenidos
recetas = pd.DataFrame({
    'Titulo': titulo_receta,
    'Descripcion': descripcion,
    'Ingredientes': ingredientes,
    'Instrucciones': instrucciones
})

# Limpiar el DataFrame
recetas = recetas.dropna()

# Guardar el DataFrame en un archivo CSV y Excel
recetas.to_csv('Recetas.csv', index=False)

# Asegurarse de que el módulo openpyxl esté instalado
try:
    import openpyxl
    recetas.to_excel('Recetas.xlsx', index=False)
except ImportError:
    print("El módulo 'openpyxl' no está instalado. Ejecute 'pip install openpyxl' para instalarlo.")


Enlaces obtenidos: []


,Enlaces


Progreso: 0it [00:00, ?it/s]


,Titulo


,Descripcion


,Ingredientes


,Instrucciones
